In [2]:
import glob
from sys import exit, path
from os.path import join, expanduser, exists

import numpy as np
import pandas as pd
import scipy.interpolate as spi
import scipy.signal as sps

from bokeh.io import output_file, export_png, export_svgs, show, output_notebook
from bokeh.transform import linear_cmap
from bokeh.plotting import figure
from bokeh.models import ColorBar, ColumnDataSource, Span
from bokeh.layouts import gridplot
import bokeh.palettes
import colorcet as cc

path.insert(1, expanduser('~/src/noexiit/software/analyses'))

output_notebook()

Loading BokehJS ...

In [3]:
from analyze_fictrac import parse_dats, unconcat_df

pson_open_loop = parse_dats("/mnt/2TB/data_in/HK_20200317/pson_open_loop/", 1, 5, "offline")
dcor_open_loop = parse_dats("/mnt/2TB/data_in/HK_20200316/dcor_open_loop/", 1, 5, "offline")

The ball_radius argument must be in mm. Confirm by inputting 'y'. Otherwise, hit any other key to quit. y
The ball_radius argument must be in mm. Confirm by inputting 'y'. Otherwise, hit any other key to quit. y


In [4]:
pson_open_loop_list = unconcat_df(pson_open_loop)
dcor_open_loop_list = unconcat_df(dcor_open_loop)

# Stimulus analyses

**Goal:** I have the beetle position relative to the ball map, and the ant position relative to the beetle position. What I want is to show the ant position relative to the ball map, as well as the beetle position relative to the ball map. I.e. I want to show beetle and ant movements relative to the same frame of reference.

In [5]:
stims_pson_open_loop = sorted(glob.glob("/mnt/2TB/data_in/HK_20200317/pson_open_loop/*/stimulus/*.csv"))
stims_pson_open_loop_list = [pd.read_csv(stim) for stim in stims_pson_open_loop]

To work through a putative set of analyses, let's platy around with **one pair** of corresponding dataframes:

In [6]:
stims_pson_open_loop_list[6].tail()

,Elapsed time,Calendar time,Stepper output (degs),Servo output (degs)
12327,300.318597,"""2020_03_17, 21:45:52""",0.0,0
12328,300.338611,"""2020_03_17, 21:45:52""",0.0,0
12329,300.358671,"""2020_03_17, 21:45:52""",0.0,0
12330,300.378678,"""2020_03_17, 21:45:52""",0.0,0
12331,300.398639,"""2020_03_17, 21:45:52""",0.0,0


In [7]:
pson_open_loop_list[6].tail()

,frame_cntr,delta_rotn_vector_cam_x,delta_rotn_vector_cam_y,delta_rotn_vector_cam_z,delta_rotn_err_score,delta_rotn_vector_lab_x,delta_rotn_vector_lab_y,delta_rotn_vector_lab_z,abs_rotn_vector_cam_x,abs_rotn_vector_cam_y,...,delta_timestamp,alt_timestamp,secs_elapsed,framerate_hz,mins_elapsed,min_int,X_mm,Y_mm,speed_mm_s,animal
31045,31046,0.062970,-0.017149,0.002135,4188.847080,-0.011718,-0.063689,-0.008380,0.834680,1.296667,...,9.992097,3.165305e+07,310.224638,100.079092,5.170411,6,-573.399987,124.643746,32.404724,6
31046,31047,0.058911,-0.021622,0.007231,4384.587598,-0.018553,-0.059978,-0.006980,0.881642,1.298039,...,9.992097,3.165305e+07,310.234630,100.079092,5.170577,6,-573.090385,124.695573,31.415849,6
31047,31048,0.040849,-0.038811,0.018950,4071.402769,-0.040251,-0.043027,-0.007908,0.906706,1.281921,...,9.992097,3.165306e+07,310.244622,100.079092,5.170744,6,-572.807165,124.614505,29.482813,6
31048,31049,0.047433,-0.017704,0.007985,3936.316075,-0.016465,-0.048368,-0.004063,0.942635,1.281751,...,9.992097,3.165306e+07,310.254614,100.079092,5.170910,6,-572.554609,124.652978,25.567144,6
31049,31050,0.048460,-0.018063,0.019465,4384.587598,-0.023554,-0.049744,0.004928,0.971422,1.285879,...,9.992097,3.165306e+07,310.264606,100.079092,5.171077,6,-572.279602,124.663052,27.540959,6


In [8]:
stim_df = stims_pson_open_loop_list[6]
fictrac_df = pson_open_loop_list[6]

We begin by interpolating from the 0 to 180 degrees range of the servo output, to real world values of 0 to 27 mm, which is the physical range of the servo output. We know this relationship is linear. Remember, this distance is the distance the servo, i.e. stimulus traveled. 

In [9]:
# Generate interpolation function:
f_servo = spi.interp1d(np.linspace(0,180),np.linspace(0,27))

# Apply function:
stim_df["Servo output (mm)"] =  f_servo(stim_df["Servo output (degs)"])
stim_df.tail()

,Elapsed time,Calendar time,Stepper output (degs),Servo output (degs),Servo output (mm)
12327,300.318597,"""2020_03_17, 21:45:52""",0.0,0,0.0
12328,300.338611,"""2020_03_17, 21:45:52""",0.0,0,0.0
12329,300.358671,"""2020_03_17, 21:45:52""",0.0,0,0.0
12330,300.378678,"""2020_03_17, 21:45:52""",0.0,0,0.0
12331,300.398639,"""2020_03_17, 21:45:52""",0.0,0,0.0


We next need to align our dataframes.
First, rename `Elapsed time` in the `stim_df` so it shares the column name `secs_elapsed` with the `fictrac_df`.

In [10]:
stim_df = stim_df.rename(columns={"Elapsed time": "secs_elapsed"})

We now need to merge the two dataframes according to the common `secs_elapsed` column we just made. We are going to use the `merge_ordered` method, with the `fill_method` set to `ffill`, which means "forward fill". Forward fill takes care of `NaN` values by propagating the last valid observation forward.

In [11]:
df_merged = pd.merge_ordered(stim_df, fictrac_df, on="secs_elapsed", fill_method="ffill")
df_merged.tail() 

,secs_elapsed,Calendar time,Stepper output (degs),Servo output (degs),Servo output (mm),frame_cntr,delta_rotn_vector_cam_x,delta_rotn_vector_cam_y,delta_rotn_vector_cam_z,delta_rotn_err_score,...,seq_cntr,delta_timestamp,alt_timestamp,framerate_hz,mins_elapsed,min_int,X_mm,Y_mm,speed_mm_s,animal
43377,310.224638,"""2020_03_17, 21:45:52""",0.0,0.0,0.0,31046,0.062970,-0.017149,0.002135,4188.847080,...,31046,9.992097,3.165305e+07,100.079092,5.170411,6,-573.399987,124.643746,32.404724,6
43378,310.234630,"""2020_03_17, 21:45:52""",0.0,0.0,0.0,31047,0.058911,-0.021622,0.007231,4384.587598,...,31047,9.992097,3.165305e+07,100.079092,5.170577,6,-573.090385,124.695573,31.415849,6
43379,310.244622,"""2020_03_17, 21:45:52""",0.0,0.0,0.0,31048,0.040849,-0.038811,0.018950,4071.402769,...,31048,9.992097,3.165306e+07,100.079092,5.170744,6,-572.807165,124.614505,29.482813,6
43380,310.254614,"""2020_03_17, 21:45:52""",0.0,0.0,0.0,31049,0.047433,-0.017704,0.007985,3936.316075,...,31049,9.992097,3.165306e+07,100.079092,5.170910,6,-572.554609,124.652978,25.567144,6
43381,310.264606,"""2020_03_17, 21:45:52""",0.0,0.0,0.0,31050,0.048460,-0.018063,0.019465,4384.587598,...,31050,9.992097,3.165306e+07,100.079092,5.171077,6,-572.279602,124.663052,27.540959,6


But see how the last values are longer than acceptable propagations of the last valid observation from the shorter dataframe? 
Performing the forward fill propagated the _last_ valid observation for a longer than acceptable amount of time. We need to truncate the longer dataframe so that the entries on its last row is as close to the entries on the last row of the smaller dataframe. For example, if one dataframe recorded for 96 seconds, and the other for 100 seconds, then the forward fill method will propagate in the merged dataframe, the value from the 96th second all the way to the 100 second mark. That's 4 whole seconds of propagation, which is a long time, especially if our sampling frequency is much shorter than that. We handle this artifact with a simple function:

In [12]:
def get_smaller_last_val_in_col(df1, df2, common_col):
    assert common_col in df1 and common_col in df2, \
        f"{df1} and {df2} do not share {common_col}"
    
    compare = float(df1[common_col].tail(1)) > float(df2[common_col].tail(1))
    if compare is True:
        return float(df2[common_col].tail(1))
    else:
        return float(df1[common_col].tail(1))

In [13]:
smaller_last_val = get_smaller_last_val_in_col(stim_df, fictrac_df, "secs_elapsed")
smaller_last_val 

300.3986387252808

In [14]:
# Drop anything larger than the smaller last val:
df_merged_trunc = df_merged[df_merged.secs_elapsed < smaller_last_val]
df_merged_trunc.tail()

,secs_elapsed,Calendar time,Stepper output (degs),Servo output (degs),Servo output (mm),frame_cntr,delta_rotn_vector_cam_x,delta_rotn_vector_cam_y,delta_rotn_vector_cam_z,delta_rotn_err_score,...,seq_cntr,delta_timestamp,alt_timestamp,framerate_hz,mins_elapsed,min_int,X_mm,Y_mm,speed_mm_s,animal
42388,300.362438,"""2020_03_17, 21:45:52""",0.0,0.0,0.0,30059,0.059656,-0.016341,0.010192,3875.662252,...,30059,9.992097,3.164939e+07,100.079092,5.006041,6,-562.064475,116.811407,31.360214,6
42389,300.372430,"""2020_03_17, 21:45:52""",0.0,0.0,0.0,30060,0.048314,-0.027941,0.023703,3679.921734,...,30060,9.992097,3.164939e+07,100.079092,5.006207,6,-562.108847,117.111082,30.318147,6
42390,300.378678,"""2020_03_17, 21:45:52""",0.0,0.0,0.0,30060,0.048314,-0.027941,0.023703,3679.921734,...,30060,9.992097,3.164939e+07,100.079092,5.006207,6,-562.108847,117.111082,30.318147,6
42391,300.382422,"""2020_03_17, 21:45:52""",0.0,0.0,0.0,30061,0.049508,-0.025624,0.022875,4541.180012,...,30061,9.992097,3.164940e+07,100.079092,5.006374,6,-562.164888,117.406689,30.111073,6
42392,300.392414,"""2020_03_17, 21:45:52""",0.0,0.0,0.0,30062,0.055811,-0.012640,0.010987,3613.090909,...,30062,9.992097,3.164940e+07,100.079092,5.006540,6,-562.304307,117.662421,29.149770,6


Recall that the `servo output` is the amount the servo has extended from its initial position. In other words, by itself, `servo output` is not technically enough information to tell us how close or far the linear servo, i.e. ant stimulus, is from the beetle on the ball. For example, if the linear servo + ant tether was positioned very far from the beetle on the ball, then even at the linear servo's full extension, the ant will not touch the beetle. In other words, we have to make an approximation of how far the ant stimulus is from the beetle, when linear servo is fully extended. Making this measurement with both insects tethered is ... not practical, especially without preemptively releasing interaction behaviours. We instead have to make a simplifying assumption--that the full extension of the linear servo is approximately close enough to exactly make contact between the tethered beetle and the presented ant stimulus. From experimental observation, this assumption is reasonable, but the exact value will differ from trial to trial. I should think about a way of consistently measuring this distance so I that future experiments are more reproducible. 

If we assume the linear servo's full extension of 27 mm is when the ant exactly makes contact with the beetle, we can calculate how far the ant is from the tethered beetle:

\begin{equation}
\text{distance apart (mm)} = \text{27 mm} - \text{servo output (mm)}
\end{equation}

In [15]:
df_merged_trunc["dist_from_stim"] = 27 - df_merged_trunc["Servo output (mm)"]
df_merged_trunc.tail(3)

/home/hank-x299/anaconda3/envs/with_plotly/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,secs_elapsed,Calendar time,Stepper output (degs),Servo output (degs),Servo output (mm),frame_cntr,delta_rotn_vector_cam_x,delta_rotn_vector_cam_y,delta_rotn_vector_cam_z,delta_rotn_err_score,...,delta_timestamp,alt_timestamp,framerate_hz,mins_elapsed,min_int,X_mm,Y_mm,speed_mm_s,animal,dist_from_stim
42390,300.378678,"""2020_03_17, 21:45:52""",0.0,0.0,0.0,30060,0.048314,-0.027941,0.023703,3679.921734,...,9.992097,3.164939e+07,100.079092,5.006207,6,-562.108847,117.111082,30.318147,6,27.0
42391,300.382422,"""2020_03_17, 21:45:52""",0.0,0.0,0.0,30061,0.049508,-0.025624,0.022875,4541.180012,...,9.992097,3.164940e+07,100.079092,5.006374,6,-562.164888,117.406689,30.111073,6,27.0
42392,300.392414,"""2020_03_17, 21:45:52""",0.0,0.0,0.0,30062,0.055811,-0.012640,0.010987,3613.090909,...,9.992097,3.164940e+07,100.079092,5.006540,6,-562.304307,117.662421,29.149770,6,27.0


My stepper motor homes to a reed switch and sets that home position as position 0. We can see from the dataset's initial observation below that the stepper position indeed starts at 0 degrees:

In [16]:
df_merged_trunc.head(3)

,secs_elapsed,Calendar time,Stepper output (degs),Servo output (degs),Servo output (mm),frame_cntr,delta_rotn_vector_cam_x,delta_rotn_vector_cam_y,delta_rotn_vector_cam_z,delta_rotn_err_score,...,delta_timestamp,alt_timestamp,framerate_hz,mins_elapsed,min_int,X_mm,Y_mm,speed_mm_s,animal,dist_from_stim
0,0.019984,NaN,NaN,NaN,NaN,1,0.038689,-0.032769,-0.016849,5907.113122,...,9.992097,3.153617e+07,NaN,0.000333,1,-0.198959,0.067479,NaN,6,NaN
1,0.022198,"""2020_03_17, 21:40:51""",0.0,0.0,0.0,1,0.038689,-0.032769,-0.016849,5907.113122,...,9.992097,3.153617e+07,NaN,0.000333,1,-0.198959,0.067479,NaN,6,27.0
2,0.029976,"""2020_03_17, 21:40:51""",0.0,0.0,0.0,2,0.058906,-0.029490,-0.000025,5703.295210,...,9.992097,3.153618e+07,100.079092,0.000500,1,-0.503488,0.157438,31.77933,6,27.0


From watching the FicTrac video, we can tell that when `stepper output (degs)` is 0, the beetle and ant are pointed in the same direction in true physical space. Therefore, the initial heading frame of reference of "0", is the same for both the beetle and the ant. In addition, `Stepper output (degs)` _**increases**_ in value when the stepper moves _**clockwise**_; clockwise here assumes that the observer is looking at the rig from the top-down. 

In [17]:
# TODO: MAKE A MARKDOWN CELL EXPLAINING THE TRIG IN DETAIL

In [18]:
df_merged_trunc['stim_X_mm'] = df_merged_trunc.apply(lambda row: (row["X_mm"] + (row["dist_from_stim"] * np.cos(np.deg2rad(row["Stepper output (degs)"])))), axis=1)
df_merged_trunc['stim_Y_mm'] = df_merged_trunc.apply(lambda row: (row["Y_mm"] + (row["dist_from_stim"] * np.sin(np.deg2rad(row["Stepper output (degs)"])))), axis=1)

/home/hank-x299/anaconda3/envs/with_plotly/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/hank-x299/anaconda3/envs/with_plotly/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


We check that we get reasonable values for the stimulus X and Y coordinates:

In [19]:
df_merged_trunc.loc[df_merged_trunc["Stepper output (degs)"] > 0.42].head(3)

,secs_elapsed,Calendar time,Stepper output (degs),Servo output (degs),Servo output (mm),frame_cntr,delta_rotn_vector_cam_x,delta_rotn_vector_cam_y,delta_rotn_vector_cam_z,delta_rotn_err_score,...,framerate_hz,mins_elapsed,min_int,X_mm,Y_mm,speed_mm_s,animal,dist_from_stim,stim_X_mm,stim_Y_mm
8235,58.303617,"""2020_03_17, 21:41:50""",0.421875,0.0,0.0,5833,0.026184,-0.020152,0.009667,2936.104154,...,100.079092,0.971565,1,-343.894891,-190.996611,17.096222,6,27.0,-316.895622,-190.797809
8236,58.303886,"""2020_03_17, 21:41:50""",0.421875,0.0,0.0,5834,0.025118,-0.018025,0.006210,3407.537026,...,100.079092,0.971731,1,-344.046080,-190.962261,15.516538,6,27.0,-317.046812,-190.763459
8237,58.313878,"""2020_03_17, 21:41:50""",0.421875,0.0,0.0,5835,0.040620,-0.015219,-0.007514,3416.796508,...,100.079092,0.971898,1,-344.247759,-191.008501,20.707755,6,27.0,-317.248491,-190.809699


In [20]:
from analyze_stimulus import plot_fictrac_XY_with_stim

df_plot = df_merged_trunc
# df_plot = df_plot[::5]
# plot_fictrac_XY_with_stim(df_plot, high_percentile=97, alpha=0.06, size=2) 

## Alternative fill method: interpolation

Instead of doing a forward fill method where we propagate the last valid observation, we can make an interpolation of what the NaN values might be. 

In [21]:
df_merged_no_fill = pd.merge_ordered(stim_df, fictrac_df, on="secs_elapsed", fill_method=None)

# Drop anything larger than the smaller last val:
df_merged_no_fill_trunc = df_merged_no_fill[df_merged_no_fill.secs_elapsed < smaller_last_val]
df_merged_no_fill_trunc.tail()

,secs_elapsed,Calendar time,Stepper output (degs),Servo output (degs),Servo output (mm),frame_cntr,delta_rotn_vector_cam_x,delta_rotn_vector_cam_y,delta_rotn_vector_cam_z,delta_rotn_err_score,...,seq_cntr,delta_timestamp,alt_timestamp,framerate_hz,mins_elapsed,min_int,X_mm,Y_mm,speed_mm_s,animal
42388,300.362438,NaN,NaN,NaN,NaN,30059.0,0.059656,-0.016341,0.010192,3875.662252,...,30059.0,9.992097,3.164939e+07,100.079092,5.006041,6.0,-562.064475,116.811407,31.360214,6
42389,300.372430,NaN,NaN,NaN,NaN,30060.0,0.048314,-0.027941,0.023703,3679.921734,...,30060.0,9.992097,3.164939e+07,100.079092,5.006207,6.0,-562.108847,117.111082,30.318147,6
42390,300.378678,"""2020_03_17, 21:45:52""",0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42391,300.382422,NaN,NaN,NaN,NaN,30061.0,0.049508,-0.025624,0.022875,4541.180012,...,30061.0,9.992097,3.164940e+07,100.079092,5.006374,6.0,-562.164888,117.406689,30.111073,6
42392,300.392414,NaN,NaN,NaN,NaN,30062.0,0.055811,-0.012640,0.010987,3613.090909,...,30062.0,9.992097,3.164940e+07,100.079092,5.006540,6.0,-562.304307,117.662421,29.149770,6


We use a linear interpolation. Understand that the beginning `NaN` values will not be interpolated, because those values are not preceded by valid observations, they are only proceeded by valid observations. In addition, non-numeric values will not be interpolated. We can fill those `NaN`s with a forward fill. One extra caveat to address is found in [this Stack Exchange post](https://stackoverflow.com/questions/58843656/pandas-interpolation-valueerrorinvalid-fill-method-expecting-pad-ffill-or). We need to handle the interpolation carefully.

In [22]:
df_merged_trunc_interp = df_merged_no_fill_trunc.interpolate(method="linear")
df_merged_trunc_interp.tail()

,secs_elapsed,Calendar time,Stepper output (degs),Servo output (degs),Servo output (mm),frame_cntr,delta_rotn_vector_cam_x,delta_rotn_vector_cam_y,delta_rotn_vector_cam_z,delta_rotn_err_score,...,seq_cntr,delta_timestamp,alt_timestamp,framerate_hz,mins_elapsed,min_int,X_mm,Y_mm,speed_mm_s,animal
42388,300.362438,NaN,0.0,0.0,0.0,30059.0,0.059656,-0.016341,0.010192,3875.662252,...,30059.0,9.992097,3.164939e+07,100.079092,5.006041,6.0,-562.064475,116.811407,31.360214,6
42389,300.372430,NaN,0.0,0.0,0.0,30060.0,0.048314,-0.027941,0.023703,3679.921734,...,30060.0,9.992097,3.164939e+07,100.079092,5.006207,6.0,-562.108847,117.111082,30.318147,6
42390,300.378678,"""2020_03_17, 21:45:52""",0.0,0.0,0.0,30060.5,0.048911,-0.026782,0.023289,4110.550873,...,30060.5,9.992097,3.164940e+07,100.079092,5.006290,6.0,-562.136867,117.258886,30.214610,NaN
42391,300.382422,NaN,0.0,0.0,0.0,30061.0,0.049508,-0.025624,0.022875,4541.180012,...,30061.0,9.992097,3.164940e+07,100.079092,5.006374,6.0,-562.164888,117.406689,30.111073,6
42392,300.392414,NaN,0.0,0.0,0.0,30062.0,0.055811,-0.012640,0.010987,3613.090909,...,30062.0,9.992097,3.164940e+07,100.079092,5.006540,6.0,-562.304307,117.662421,29.149770,6


In [ ]:
df_merged_trunc_interp = df_merged_trunc_interp.ffill(axis=0)
df_merged_trunc_interp.tail()

We proceed with the rest of the pipeline:

In [ ]:
df_merged_trunc_interp["dist_from_stim"] = 27 - df_merged_trunc_interp["Servo output (mm)"]

df_merged_trunc_interp['stim_X_mm'] = df_merged_trunc_interp.apply(lambda row: (row["X_mm"] + (row["dist_from_stim"] * np.cos(np.deg2rad(row["Stepper output (degs)"])))), axis=1)
df_merged_trunc_interp['stim_Y_mm'] = df_merged_trunc_interp.apply(lambda row: (row["Y_mm"] + (row["dist_from_stim"] * np.sin(np.deg2rad(row["Stepper output (degs)"])))), axis=1)

df_merged_trunc_interp.tail()

In [ ]:
from analyze_stimulus import plot_fictrac_XY_with_stim

plot_fictrac_XY_with_stim(df_merged_trunc_interp)

## Test `analyze_stimulus.py` functions:

In [ ]:
from analyze_stimulus import parse_2dof_stimulus, merge_stimulus_with_data, make_stimulus_trajectory, plot_fictrac_XY_with_stim

# Use Platyusa open loop data as test:
stims = parse_2dof_stimulus("/mnt/2TB/data_in/HK_20200317/pson_open_loop/", 1, 0, 27, 27)
merged = merge_stimulus_with_data(stims, pson_open_loop, fill_method="linear")
stimmed = make_stimulus_trajectory(merged)

# Pull one out and plot:
stimmed_6 = unconcat_df(stimmed)[6]
plot_fictrac_XY_with_stim(stimmed_6, high_percentile=97, alpha=0.06, size=2) 